In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yashm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yashm\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('Datasets\ChatBot_Dataset\intents.json').read()
intents = json.loads(data_file)


In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [5]:
pickle.dump(words,open('Datasets\ChatBot_Dataset\words.pkl','wb'))
pickle.dump(classes,open('Datasets\ChatBot_Dataset\classes.pkl','wb'))

In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


C:\Users\yashm\AppData\Local\Temp\ipykernel_42936\474059937.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [8]:

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200


C:\Users\yashm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


10/10 [==============================] - 0s 3ms/step - loss: 2.2744 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1296 - accuracy: 0.1702
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0914 - accuracy: 0.1702
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9488 - accuracy: 0.2340
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8010 - accuracy: 0.4681
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7811 - accuracy: 0.2979
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6648 - accuracy: 0.5745
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3794 - accuracy: 0.6170
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3132 - accuracy: 0.5532
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.2781 - accuracy: 0.6170
Epoch 11/200
10/10 [=

10/10 [==============================] - 0s 2ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0456 - accuracy: 0.9787
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0293 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0762 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0375 - accuracy: 0.9787
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0977 - accuracy: 0.9787
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0621 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0279 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0273 - accuracy: 1.0000
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.0769 - accuracy: 0.9787
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0426 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0272 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0172 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0166 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0261 - accuracy: 1.0000
Epoc